# 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from geopy.distance import geodesic
import matplotlib as mpl
import tqdm
from scipy.spatial.distance import cdist
from scipy.spatial import cKDTree
from sklearn.cluster import KMeans
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings("ignore")

# 데이터 불러오기

In [2]:
train_data = pd.read_csv("/data/ephemeral/home/house/data/train.csv")
test_data = pd.read_csv("/data/ephemeral/home/house/data/test.csv")
sample_submission = pd.read_csv("/data/ephemeral/home/house/data/sample_submission.csv")
interest_data = pd.read_csv("/data/ephemeral/home/house/data/interestRate.csv")
park_data = pd.read_csv("/data/ephemeral/home/house/data/parkInfo.csv")
school_data = pd.read_csv("/data/ephemeral/home/house/data/schoolinfo.csv")
subway_data = pd.read_csv("/data/ephemeral/home/house/data/subwayInfo.csv")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
sample_submission.head()

In [ ]:
interest_data.head()

In [ ]:
park_data.head()

In [ ]:
school_data.head()

In [ ]:
subway_data.head()

In [ ]:
train_data.dtypes

# 결측치 확인

In [ ]:
print("Train 데이터 결측치 확인:\n", train_data.isnull().sum())
print("Test 데이터 결측치 확인:\n", test_data.isnull().sum())
print("Subway 데이터 결측치 확인:\n", subway_data.isnull().sum())
print("Interest Rate 데이터 결측치 확인:\n", interest_data.isnull().sum())
print("School 데이터 결측치 확인:\n", school_data.isnull().sum())
print("Park 데이터 결측치 확인:\n", park_data.isnull().sum())

# 데이터 병합

### interestRate.csv와 병합

In [12]:
# train_data와 interest_data 병합
train_data['contract_year_month'] = train_data['contract_year_month'].astype(str)
interest_data['year_month'] = interest_data['year_month'].astype(str)

# left join을 통해 금리 데이터를 병합
train_data = pd.merge(train_data, interest_data, how='left', left_on='contract_year_month', right_on='year_month')

# 중복된 열 제거
train_data.drop(columns=['year_month'], inplace=True)

In [ ]:
train_data

### subwayinfo.csv와 병합

In [14]:
def find_closest_distance_kdtree(train_data, loc_df):
    train_coords = train_data[['latitude', 'longitude']].values
    park_coords = loc_df[['latitude', 'longitude']].values
    tree = cKDTree(park_coords)
    distances, _ = tree.query(train_coords)
    return distances

In [15]:
train_data['closest_subway_distance'] = find_closest_distance_kdtree(train_data, subway_data)

In [ ]:
train_data['closest_subway_distance']

### schoolinfo.csv와 병합

In [17]:
train_data['closest_school_distance'] = find_closest_distance_kdtree(train_data, school_data)

### parkinfo.csv와 병합

In [18]:
train_data['closest_park_distance'] = find_closest_distance_kdtree(train_data, park_data)

In [ ]:
print(train_data[['closest_subway_distance', 'closest_school_distance', 'closest_park_distance']].head())

# 데이터 분포 확인

### 지하철 위치 시각화

In [ ]:
# 지하철 위치 시각화
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# 지도에 육지와 강을 추가
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.RIVERS)

# 지하철 위치 플로팅
gdf_subway = gpd.GeoDataFrame(subway_data, geometry=gpd.points_from_xy(subway_data.longitude, subway_data.latitude))
gdf_subway.plot(ax=ax, color='blue', markersize=5)

plt.title('Subway Locations')
plt.show()

In [ ]:
# 히스토그램
train_data.hist(bins=20, figsize=(14, 10), color='skyblue')
plt.tight_layout()
plt.show()

In [ ]:
# 박스플롯 (Outlier 확인)
plt.figure(figsize=(12, 6))
sns.boxplot(data=train_data, palette='Set2')
plt.title('Boxplot of Variables')
plt.show()

# 상관관계 분석

In [ ]:
# 상관관계 히트맵
plt.figure(figsize=(10, 8))
sns.heatmap(train_data.corr(), annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.2)
plt.title('Correlation Heatmap')
plt.show()

### 전세가와 금리 분포 확인 및 비교

In [ ]:
# 전세가 분포 시각화
plt.figure(figsize=(10, 6))
sns.histplot(train_data['deposit'], kde=True, bins=30)
plt.title('Deposit Distribution')
plt.xlabel('Deposit')
plt.ylabel('Frequency')
plt.show()

# 금리 분포 시각화
plt.figure(figsize=(10, 6))
sns.histplot(train_data['interest_rate'], kde=True, color='red', bins=30)
plt.title('Interest Rate Distribution')
plt.xlabel('Interest Rate')
plt.ylabel('Frequency')
plt.show()

### 계약 유형 별 전세가 비교

In [ ]:
# 계약 유형별 전세가 상자 그림
plt.figure(figsize=(10, 6))
sns.boxplot(x='contract_type', y='deposit', data=train_data)
plt.title('Deposit by Contract Type')
plt.xlabel('Contract Type')
plt.ylabel('Deposit')
plt.show()

### 건축 연도 별 전세가 비교

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='built_year', y='deposit', data=train_data)
plt.title('Deposit by Built Year')
plt.xlabel('Built Year')
plt.ylabel('Deposit')
plt.show()

### 층수별 전세가 비교

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='floor', y='deposit', data=train_data)
plt.title('Deposit by Floor')
plt.xlabel('Floor')
plt.ylabel('Deposit')
plt.show()


### 면적과 전세가 비교

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='area_m2', y='deposit', data=train_data)
plt.title('Deposit by Area (m²)')
plt.xlabel('Area (m²)')
plt.ylabel('Deposit')
plt.show()


### latitude와 longitude를 활용한 전세가 지리적 분포 분석

In [ ]:
# 위도와 경도를 기반으로 한 전세가 분포 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(x='longitude', y='latitude', hue='deposit', data=train_data, palette='coolwarm', alpha=0.6)
plt.title('Geographical Distribution of Deposit')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# 전세가 데이터를 GeoDataFrame으로 변환
gdf_deposit = gpd.GeoDataFrame(train_data, geometry=gpd.points_from_xy(train_data.longitude, train_data.latitude))

# 전세가 값에 따라 색상을 지정하기 위한 컬러맵 설정
norm = plt.Normalize(vmin=gdf_deposit['deposit'].min(), vmax=gdf_deposit['deposit'].max())
cmap = plt.cm.get_cmap('coolwarm')

# 지도 생성
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# 지도에 육지, 해안선, 국경, 강 추가
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.RIVERS)

# 전세가에 따라 위치에 플로팅
gdf_deposit.plot(ax=ax, color=gdf_deposit['deposit'].apply(lambda x: cmap(norm(x))), markersize=5)

plt.title('Geographical Distribution of Deposit')
plt.show()

### 근접성 변수와 전세가의 관계

In [ ]:
# 지하철과의 거리 vs 전세가
plt.figure(figsize=(10, 6))
sns.scatterplot(x='closest_subway_distance', y='deposit', data=train_data)
plt.title('Deposit by Subway Distance')
plt.xlabel('Closest Subway Distance')
plt.ylabel('Deposit')
plt.show()

In [ ]:
# 학교와의 거리 vs 전세가
plt.figure(figsize=(10, 6))
sns.scatterplot(x='closest_school_distance', y='deposit', data=train_data)
plt.title('Deposit by School Distance')
plt.xlabel('Closest School Distance')
plt.ylabel('Deposit')
plt.show()

In [ ]:
# 공원과의 거리 vs 전세가
plt.figure(figsize=(10, 6))
sns.scatterplot(x='closest_park_distance', y='deposit', data=train_data)
plt.title('Deposit by Park Distance')
plt.xlabel('Closest Park Distance')
plt.ylabel('Deposit')
plt.show()

### 거리와 전세가의 관계
지하철, 학교, 공원 거리: 각 거리와 전세가 간의 관계를 시각화한 결과
- 거리가 가까울수록 전세가가 높아지는 경향 관찰
- 즉, 접근성이 좋은 위치일수록 더 높은 전세가를 형성할 가능성 존재

# 카테고리 변수 분석

In [ ]:
# 범주형 변수의 빈도수 확인 (가상 변수 'category'로 예시)
plt.figure(figsize=(8, 6))
sns.countplot(x='contract_type', data=train_data, palette='muted')
plt.title('Count of Categories')
plt.show()

### 계약 연월별 전세가 및 금리 추세

In [34]:
train_data['contract_year_month'] = pd.to_datetime(train_data['contract_year_month'], format='%Y%m')
monthly_avg = train_data.groupby('contract_year_month').agg({'deposit': 'mean', 'interest_rate': 'mean'}).reset_index()

In [ ]:
# 계약 연월별 전세가 및 금리 추세 시각화 개선
plt.figure(figsize=(12, 8))

# 첫 번째 y축: 전세가 (Deposit) 시각화
sns.lineplot(x='contract_year_month', y='deposit', data=monthly_avg, label='Average Deposit', color='blue')

# 두 번째 y축: 금리 (Interest Rate) 시각화
ax2 = plt.twinx()  # 두 번째 y축을 추가
sns.lineplot(x='contract_year_month', y='interest_rate', data=monthly_avg, label='Average Interest Rate', color='red', ax=ax2)

# 그래프 제목과 축 설정
plt.title('Interest Rate and Deposit Over Time')
plt.xlabel('Contract Year-Month')
plt.ylabel('Deposit (Blue)', color='blue')
ax2.set_ylabel('Interest Rate (Red)', color='red')

# x축 라벨을 일정 간격으로 보기 쉽게 회전
plt.xticks(rotation=45)

# X축 간격 설정: 라벨이 겹치지 않도록 일정 간격만 표시
plt.xticks(ticks=plt.gca().get_xticks()[::6])  # 라벨을 6개마다 한 번씩 표시

# Y축 범위 설정: 금리의 변화를 더 잘 보이도록 조정
ax2.set_ylim(monthly_avg['interest_rate'].min() - 0.5,monthly_avg['interest_rate'].max() + 0.5)

plt.legend(loc="upper left")
plt.show()

In [ ]:
# 시계열 분해 (계절성 및 트렌드 파악)
result = seasonal_decompose(monthly_avg['deposit'], model='additive', period=12)
result.plot()
plt.show()

- 전세가의 전반적인 상승 추세를 확인

## 상호작용 효과 탐색

In [ ]:
# 거리와 면적의 상호작용이 전세가에 미치는 영향
plt.figure(figsize=(10, 6))
sns.scatterplot(x='area_m2', y='closest_subway_distance', hue='deposit', data=train_data, palette='coolwarm')
plt.title('Deposit by Area and Subway Distance')
plt.xlabel('Area (m²)')
plt.ylabel('closest_subway_distance')
plt.show()

In [ ]:
min(train_data['closest_subway_distance'])

- 면적이 클수록 전세가가 증가하는 경향을 확인
- 지하철과의 거리가 가까울수록 deposit 증가하는 경향을 확인

### 지역별 분석

In [ ]:
coords = train_data[['latitude', 'longitude']]
wcss = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(coords)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 20), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

### 엘보우 방법(Elbow Method)
- 클러스터 수에 따른 WCSS(Within-Cluster Sum of Squares)를 계산하고, 그래프에서 급격한 변화가 있는 지점을 찾음
- 이 지점이 적절한 클러스터 수
- 여기서는 3

In [ ]:
# 위도와 경도를 기반으로 KMeans 클러스터링
coords = train_data[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=3, random_state=42).fit(coords)
train_data['region'] = kmeans.labels_

# 지역별 전세가 분포 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='region', y='deposit', data=train_data)
plt.title('Deposit Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Deposit')
plt.show()

In [ ]:
# 클러스터별 색상 지정
colors = {0: 'red', 1: 'blue', 2: 'green'}  # 3개의 클러스터에 각각 색상 지정

# 시각화
plt.figure(figsize=(10, 6))

# 각 클러스터에 해당하는 점들을 색상별로 플롯
for cluster, color in colors.items():
    clustered_data = train_data[train_data['region'] == cluster]
    plt.scatter(clustered_data['longitude'], clustered_data['latitude'], 
                c=color, label=f'Region {cluster}', alpha=0.5, s=10)

# 제목 및 축 레이블 설정
plt.title('KMeans Clustering of Regions (Based on Latitude and Longitude)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# 범례 추가
plt.legend()

# 시각화 표시
plt.show()

# Feature Engineering

### 모델링에 필요한 라이브러리 임포트

In [28]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
import optuna

In [29]:
# 날짜 관련 피처 생성
train_data['contract_year_month'] = pd.to_datetime(train_data['contract_year_month'], format='%Y%m')
train_data['contract_year'] = train_data['contract_year_month'].dt.year
train_data['contract_month'] = train_data['contract_year_month'].dt.month
train_data['contract_season'] = train_data['contract_year_month'].dt.quarter

In [30]:
# 거리 로그 변환
train_data['log_subway_distance'] = np.log1p(train_data['closest_subway_distance'])
train_data['log_school_distance'] = np.log1p(train_data['closest_school_distance'])
train_data['log_park_distance'] = np.log1p(train_data['closest_park_distance'])

In [31]:
# 피처 및 타겟 설정
X = train_data[['area_m2', 'floor', 'built_year', 'region', 'latitude', 'longitude', 'log_subway_distance', 'log_school_distance', 'log_park_distance', 'contract_year', 'contract_month', 'contract_season']]
y = train_data['deposit']

In [32]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeling

In [33]:
# 모델 정의
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    #'RandomForest': RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42),
    #'GradientBoosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    'XGBoost': xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0, random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42)
}

### 기본 모델 적용 및 평가(K-Fold CV)

In [34]:
# KFold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# 모델 학습 및 평가
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
    print(f"{name}: Mean MAE = {-scores.mean():.4f}, Std = {scores.std():.4f}")

### 하이퍼파라미터 튜닝(Optuna)

In [ ]:
# def rf_objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 50, 300)
#     max_depth = trial.suggest_int('max_depth', 3, 20)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
#     model = RandomForestRegressor(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         random_state=42
#     )
    
#     score = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
#     return -score.mean()

# rf_study = optuna.create_study(direction='minimize')
# rf_study.optimize(rf_objective, n_trials=50)
# print("Best parameters for RandomForest: ", rf_study.best_params)

In [ ]:
def xgb_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        tree_method='gpu_hist',
        gpu_id=0,
        random_state=42
    )
    
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
    return -score.mean()

xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(xgb_objective, n_trials=50)
print("Best parameters for XGBoost: ", xgb_study.best_params)

In [ ]:
def lgb_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', -1, 20) # -1 means no limit
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    num_leaves = trial.suggest_int('num_leaves', 2, 256)
    
    model = lgb.LGBMRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        random_state=42
    )
    
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
    return -score.mean()

lgb_study = optuna.create_study(direction='minimize')
lgb_study.optimize(lgb_objective, n_trials=50)
print("Best parameters for LightGBM: ", lgb_study.best_params)

# 앙상블 모델(Voting Regressor)

In [41]:
xgb_tmp_best_params = {'n_estimators': 267, 'max_depth': 20, 'learning_rate': 0.03474061231102679, 'subsample': 0.7137831707849674}

In [ ]:
## 최적화된 하이퍼파라미터를 반영한 모델 정의
#rf_best = RandomForestRegressor(**rf_study.best_params, random_state=42)
xgb_best = xgb.XGBRegressor(**xgb_tmp_best_params, tree_method='gpu_hist', gpu_id=0, random_state=42)
#lgb_best = lgb.LGBMRegressor(**lgb_study.best_params, random_state=42)
# voting_regressor = VotingRegressor(estimators=[
#     #('rf', rf_best),
#     ('xgb', xgb_best),
#     ('lgb', lgb_best)
# ])

voting_scores = cross_val_score(xgb_best, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
print(f"Voting Regressor: Mean MAE = {-voting_scores.mean():.4f}, Std = {voting_scores.std():.4f}")

In [ ]:
model_names = ['XGBoost', 'LightGBM', 'Voting Regressor']
model_mae = [
    #-rf_study.best_value,
    xgb_study.best_value,
    lgb_study.best_value,
    -voting_scores.mean()
]

plt.figure(figsize=(10, 6))
plt.bar(model_names, model_mae, color=['green', 'orange', 'red'])
plt.ylabel('MAE')
plt.title('Model Comparison')
plt.show()

In [ ]:
# 5. Voting Regressor로 예측 수행
xgb_best.fit(X_train, y_train)
y_pred = xgb_best.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Final Model MAE: {mae:.4f}")

In [ ]:
# 6. 결과 분석 및 시각화
# 실제값과 예측값 비교
plt.figure(figsize=(10, 6))
plt.plot(y_test, y_pred, alpha=0.3, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.xlabel('Actual Deposit')
plt.ylabel('Predicted Deposit')
plt.title('Actual vs Predicted Deposit')
plt.show()

# 실제 test.csv를 활용하여 output.csv 만들기

In [45]:
# test_data와 interest_data 병합
test_data['contract_year_month'] = test_data['contract_year_month'].astype(str)
interest_data['year_month'] = interest_data['year_month'].astype(str)

# left join을 통해 금리 데이터를 병합
test_data = pd.merge(test_data, interest_data, how='left', left_on='contract_year_month', right_on='year_month')

# 중복된 열 제거
test_data.drop(columns=['year_month'], inplace=True)

In [46]:
# 거리 관련
test_data['closest_subway_distance'] = find_closest_distance_kdtree(test_data, subway_data)
test_data['closest_school_distance'] = find_closest_distance_kdtree(test_data, school_data)
test_data['closest_park_distance'] = find_closest_distance_kdtree(test_data, park_data)

In [47]:
# 날짜 관련 피처 생성
test_data['contract_year_month'] = pd.to_datetime(test_data['contract_year_month'], format='%Y%m')
test_data['contract_year'] = test_data['contract_year_month'].dt.year
test_data['contract_month'] = test_data['contract_year_month'].dt.month
test_data['contract_season'] = test_data['contract_year_month'].dt.quarter

In [48]:
# 거리 로그 변환
test_data['log_subway_distance'] = np.log1p(test_data['closest_subway_distance'])
test_data['log_school_distance'] = np.log1p(test_data['closest_school_distance'])
test_data['log_park_distance'] = np.log1p(test_data['closest_park_distance'])

In [49]:
# 위도와 경도를 기반으로 KMeans 클러스터링
coords = test_data[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=3, random_state=42).fit(coords)
test_data['region'] = kmeans.labels_

In [50]:
# 피처 및 타겟 설정
final_test = test_data[['area_m2', 'floor', 'built_year', 'region', 'latitude', 'longitude', 'log_subway_distance', 'log_school_distance', 'log_park_distance', 'contract_year', 'contract_month', 'contract_season']]

In [51]:
voting_regressor_test_pred = xgb_best.predict(final_test)
sample_submission['deposit'] = voting_regressor_test_pred
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')